In [1]:
#Per iniziare, definisco i dataframe dei libri e dei tipi dei libri.
#DOMANDA: cosa si intende per "Manifestation"?

file = "/data/SMARTDATA/books/BCT_prestiti/items.csv"
DFitems = spark.read.csv(file,header=True)
DFitems.show()

file = "/data/SMARTDATA/books/BCT_prestiti/item_media_types.csv"
DFtypes = spark.read.csv(file,header=True)
DFtypes.show()

#Come posso constatare, è possibile joinare media types e libri tramite l'id media types per vedere a quale genere
#si riferisce il libro e filtrare così i generi non necessari.

+-------+----------------+-------------+---------------+--------------+
|item_id|manifestation_id|item_media_id|home_library_id|inventory_date|
+-------+----------------+-------------+---------------+--------------+
| 288788|          324685|            Q|             27|    2007-06-04|
|1987167|           18421|            S|             13|    2013-01-28|
|2555210|          747441|            F|              2|    2016-09-15|
| 218132|            3490|            F|              2|    2008-03-12|
|1009634|          233431|            F|             19|    2004-05-27|
| 121499|          286272|            F|             27|    2010-09-01|
|2400369|          270193|            F|             16|    2016-01-27|
| 103294|          365773|            F|              2|    2010-06-08|
| 164444|          269506|            F|             20|    2006-05-23|
|2269659|          625603|            A|              3|    2015-02-20|
|1488598|          449444|            T|              2|    2011

In [2]:
#FILTRAGGIO NUMERO 1: GENERI NON NECESSARI
#Per un lettore "medio" credo sia ragionevole filtrare via i materiali multimediali (DVD, VHS, audiolibri, proiezioni
#e altri materiali elettronici), i libri in braille (che rappresentano, immagino, una forte minoranza all'interno del catalogo)
#e i materiali cartografici poiché immagino siano più usati da esperti che da lettori interessati alle raccomandazioni. Discorso
#analogo per le lettere autografe.
#Le microforme (riassunti di libri in versione compatta) credo siano più interessanti per gli studenti che, in quanto tali,
#non dovrebbero aver bisogno di raccomandazioni.
#I periodici (e tutti i magazine) non devono essere contati dal nostro sistema come libri e vanno dunque filtrati (filtraggio che
#dovrebbe anche consentirci di avere una top k più pulita).
#Per cui, post filtraggio, i generi che rimangono sono le monografie, i manoscritti e (forse) "grafica" e "oggetto". Devo
#controllare cosa si intenda con queste due categorie.
#NOTA: Come risalire ai titoli dei libri da items.csv? NOTA2: per qualche motivo, item_id e manifestation_id sono differenti

file = "/data/SMARTDATA/books/BCT_prestiti/manifestations.csv"
DFmanifestations = spark.read.csv(file,header=True)
DFmanifestations.show()

DFitems.filter(DFitems.manifestation_id == "null").show() #QUESTA RIGA MOSTRA CHE OGNI LIBRO HA IL SUO MANIFESTATION ID

#Il manifestation_id (qualunque cosa sia) è univoco?
DFitems.groupby("manifestation_id").count().agg({"count":"sum"}).show()
DFitems.select("manifestation_id").distinct().groupby("manifestation_id").count().agg({"count":"sum"}).show()


+----------------+----------------+------------+-------------+-------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|manifestation_id|edition_language|edition_date|     ISBNISSN|          EAN|               title|              author|           publisher|     loanable_since|       date_created|
+----------------+----------------+------------+-------------+-------------+--------------------+--------------------+--------------------+-------------------+-------------------+
|          202094|             ita|        2003|   8807920638|         null|Okey dokey, sono ...|   Luciani, Domenica|         Feltrinelli|               null|2003-04-22 00:00:00|
|          910309|             ita|        2017|9788893880442|         null|Che cosa ho in te...|                null|  Baldini & Castoldi|               null|2019-02-20 16:29:27|
|          382413|             ita|        2009|9788860881847|9788860881847|Morte a Firenze :...|   

In [3]:
#MANIFESTATION_ID non è univoco. Devo assolutamente capire su cosa si basa.
#UPDATE: Informandomi nella pagine di FRBR (Functional Requirements for Bibliographic Records) noto che per "manifestation"
#si intende ogni differente versione riferita allo stesso libro in item. Quindi un item può avere più manifestation_id a seconda 
#dell'edizione. Questo vuol dire che, dato che il titolo del libro è solo in manifestations.csv, per avere un titolo riferito
#ad un item_id devo joinare items e manifestations per manifestation_id e poi aggregare item_id con un groupby.

DFmanifestationitems = DFitems.join(DFmanifestations, DFitems.manifestation_id == DFmanifestations.manifestation_id).groupby("item_id", "title").count().show()


+-------+--------------------+-----+
|item_id|               title|count|
+-------+--------------------+-----+
| 176276|L' albero delle p...|    1|
| 511908|Sottovoce o mia N...|    1|
| 831805|I sette in fuga /...|    1|
| 830014|Una vecchia amica...|    1|
| 642490|L'immigrazione / ...|    1|
| 929897|Questa è la mia v...|    1|
|1182043|La nuvola Olga e ...|    1|
| 721676|Giordania : *Amma...|    1|
| 197320|Tempo di fuga : r...|    1|
| 388064|Arrivederci & Ame...|    1|
| 274535|I viaggi di Uliss...|    1|
| 388138|Interni domestici...|    1|
|  71157|Oltre la fine dei...|    1|
| 323888|XY / Sandro Veronesi|    1|
| 839057|2 : *Da Gautama B...|    1|
|2274353|Quack non fa i co...|    1|
| 308043|La Repubblica di ...|    1|
|3044711|Il bambino di Bud...|    1|
| 556793|Il Risorgimento, ...|    1|
|2569140|Caramel / un film...|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [4]:
#Risolto il mistero delle manifestations, procediamo al filtraggio dei generi. Effettuiamo un join fra items e media types per
#avere informazioni sui generi.

DFtypesitems = DFtypes.join(DFitems, DFitems.item_media_id == DFtypes.item_media_id).drop(DFitems.item_media_id)

#Controlliamo che libri sono quelli della categoria "Oggetto" e "Grafica"

DFtypesitems.where((DFtypesitems.item_media_id == "C") | (DFtypesitems.item_media_id == "L")).join(DFmanifestations, DFmanifestations.manifestation_id == DFtypesitems.manifestation_id).groupby("item_id", "title", "item_media_id").count().show()

#A quanto pare le categorie "oggetto" sono quelle basate sui magazine tematici, per cui da scartare; Le categorie "grafica" 
#sono da scartare anch'esse poiché davvero esigue rispetto al catalogo originale.

+-------+--------------------+-------------+-----+
|item_id|               title|item_media_id|count|
+-------+--------------------+-------------+-----+
|3099611|Cucina naturale :...|            L|    1|
|2396909| Le leggi illustrate|            L|    1|
|2525484|      Linux magazine|            L|    1|
|2245941|      Ruoteclassiche|            L|    1|
|2917579|        Motociclismo|            L|    1|
|2756153|Autunno / Ali Smi...|            L|    1|
|2731364|  Le idee di Susanna|            L|    1|
|2942979|Time : the weekly...|            L|    1|
|2912948|Gardenia : fiori,...|            L|    1|
|2415773|Millionaire : lib...|            L|    1|
|2912954|Gardenia : fiori,...|            L|    1|
|2929795|Camminare : per s...|            L|    1|
|2740480|Internazionale : ...|            L|    1|
|2552847|Il mio colore / F...|            C|    1|
|2637404|Sei grandi variaz...|            L|    1|
|2552846|Il mio colore / F...|            C|    1|
|2912955|Gardenia : fiori,...| 

In [5]:
DFfiltered = DFitems.filter((DFitems.item_media_id == "B") | (DFitems.item_media_id == "F"))


In [6]:
#Let's check the titles of those items through manifestations.
DFfilteredmanifestations = DFfiltered.join(DFmanifestations, DFmanifestations.manifestation_id == DFfiltered.manifestation_id).drop(DFfiltered.manifestation_id)

DFfilteredmanifestations.show()

+-------+-------------+---------------+--------------+----------------+----------------+------------+----------+----+--------------------+-------------------+--------------------+--------------+-------------------+
|item_id|item_media_id|home_library_id|inventory_date|manifestation_id|edition_language|edition_date|  ISBNISSN| EAN|               title|             author|           publisher|loanable_since|       date_created|
+-------+-------------+---------------+--------------+----------------+----------------+------------+----------+----+--------------------+-------------------+--------------------+--------------+-------------------+
| 403099|            F|             14|    2008-05-17|          100010|             ita|        1999|8879600931|null|Fiat, 1899-1999 :...|Felicioli, Riccardo|         Automobilia|          null|1999-08-05 00:00:00|
| 512772|            F|             17|    2000-05-24|          100010|             ita|        1999|8879600931|null|Fiat, 1899-1999 :...|Fe

In [7]:
#Filtraggio numero 2: amalgamare i libri uguali ma di edizioni diverse sotto un unico ID. Ciò sarà utile
#per evitare di considerare libri praticamente uguali come diversi.
#COME SI PROCEDE? Potrei joinare loans a filteredmanifestations usando item_id e poi applicare una trasformazione
#in cui, a parità di titolo, l'id venga cambiato in modo da risultare uguale.
#Cosa abbiamo in loans.csv?

file = "/data/SMARTDATA/books/BCT_prestiti/loans.csv"
DFloans = spark.read.csv(file,header=True)
DFloans.show()

#In loans abbiamo solo item_id (purtroppo) dunque dobbiamo joinare usando tale attributo E POI procedere
#a una trasformazione.

+--------------------+-------+-------------------+-------------------+-------------------+------------+----------+-----------+
|       patron_id_md5|item_id|    loan_date_begin|      loan_date_end|           due_date|from_library|to_library|end_library|
+--------------------+-------+-------------------+-------------------+-------------------+------------+----------+-----------+
|5772331ac08adcd29...| 837536|2012-06-01 10:56:48|2012-06-01 10:57:32|2012-07-01 10:56:46|          13|        13|         13|
|45d1502e068ccf492...| 353668|2012-01-10 14:55:38|2012-01-10 14:55:58|               null|          16|        27|         27|
|365f660c20c1d8cfe...|1462310|2012-02-08 15:05:34|2012-02-08 15:05:44|               null|          25|         2|          2|
|88222e7e5f8172698...| 741597|2012-02-13 17:00:09|2012-04-10 16:34:04|               null|          15|        18|         18|
|ce746e197d74b1fa7...| 161044|2012-01-12 14:19:16|2012-01-12 14:19:30|               null|          18|        

In [8]:
DFfilteredloans = DFfilteredmanifestations.join(DFloans, DFloans.item_id == DFfilteredmanifestations.item_id).drop(DFloans.item_id)
DFfilteredloans.show()

+-------+-------------+---------------+--------------+----------------+----------------+------------+-------------+-------------+--------------------+------+----------+--------------+-------------------+--------------------+-------------------+-------------------+-------------------+------------+----------+-----------+
|item_id|item_media_id|home_library_id|inventory_date|manifestation_id|edition_language|edition_date|     ISBNISSN|          EAN|               title|author| publisher|loanable_since|       date_created|       patron_id_md5|    loan_date_begin|      loan_date_end|           due_date|from_library|to_library|end_library|
+-------+-------------+---------------+--------------+----------------+----------------+------------+-------------+-------------+--------------------+------+----------+--------------+-------------------+--------------------+-------------------+-------------------+-------------------+------------+----------+-----------+
| 100010|            F|              

In [9]:
#Come sappiamo, nonostante l'item_id sia diverso per ogni singolo libro, il manifestation_id rimane lo stesso se
#la diversità deriva da una differenza di edizioni (o similari) e non di contenuto. Possiamo dunque trasformare
#l'item_id delle altre edizioni con quello della prima "facendo finta" siano tutti lo stesso libro.
#Come effettuo tale trasformazione?

DFfilteredloans.groupby("manifestation_id", "title").count().sort("count", ascending=False).show()

#Pensandoci, mi basta non considerare item_id e trattare manifestation_id come se fosse l'id originale dato che
#manifestazioni diverse dello stesso libro avranno lo stesso ID.

#Nota: Topolino "infesta" ancora il database. Fa forse parte della categoria F o B?

DFfilteredloans.filter(DFfilteredloans.title.contains("Topolino")).show()

#Nota2: pensandoci, quali libri fanno parte della categoria B?

DFfilteredloans.filter(DFfilteredloans.item_media_id =="B").show()

#Sembrano calzanti per il database filtrato (per ora)



+----------------+--------------------+-----+
|manifestation_id|               title|count|
+----------------+--------------------+-----+
|            2544|Topolino / Walt D...| 3073|
|          400318|Nel mare ci sono ...| 1525|
|          150923|                 Tex| 1358|
|          552895|Disney   Big : le...| 1282|
|          109846|L'amico ritrovato...| 1269|
|          551452|Fai bei sogni : r...| 1254|
|          147909|Paperino / Walt D...| 1169|
|          648313|1: L'amica genial...| 1118|
|          223856|Il cacciatore di ...|  961|
|          323765|Mille splendidi s...|  945|
|          548984|Il silenzio dell'...|  926|
|          351365|L'eleganza del ri...|  923|
|          188152|Dylan Dog : l'ind...|  878|
|          575600|E l'eco rispose /...|  840|
|           77302|Il signore delle ...|  821|
|          426645|L'ora di pietra :...|  796|
|          648307|[4]: L'amica geni...|  792|
|          423397|Nessuno si salva ...|  790|
|          275875|Se questo è un u

In [16]:
#Purtroppo magazine quali Topolino, Tex e Dylan Dog fanno parte della categoria "F" e vanno dunque filtrati
#caso per caso.

DFfilterednomagazines = DFfilteredloans.filter(~((DFfilteredloans.title.contains("Topolino")) | (DFfilteredloans.title.contains("Dylan Dog")) | (DFfilteredloans.title.contains("Tex")) | (DFfilteredloans.title.contains("Paperino")) | (DFfilteredloans.title.contains("Disney   Big"))))

DFfilterednomagazines.groupby("manifestation_id", "title").count().sort("count", ascending=False).show()

+----------------+--------------------+-----+
|manifestation_id|               title|count|
+----------------+--------------------+-----+
|          400318|Nel mare ci sono ...| 1525|
|          109846|L'amico ritrovato...| 1269|
|          551452|Fai bei sogni : r...| 1254|
|          648313|1: L'amica genial...| 1118|
|          223856|Il cacciatore di ...|  961|
|          323765|Mille splendidi s...|  945|
|          548984|Il silenzio dell'...|  926|
|          351365|L'eleganza del ri...|  923|
|          575600|E l'eco rispose /...|  840|
|           77302|Il signore delle ...|  821|
|          426645|L'ora di pietra :...|  796|
|          648307|[4]: L'amica geni...|  792|
|          423397|Nessuno si salva ...|  790|
|          275875|Se questo è un uo...|  787|
|          625297|[2]: L'amica geni...|  775|
|          349051|Diario di una sch...|  754|
|          420361|Il profumo delle ...|  751|
|          211449|Lo strano caso de...|  742|
|          400434|L'ultima riga de

In [24]:
#Non sembra una cattiva top 20. Adesso vorrei applicare il filtraggio numero 3: eliminare dal dataframe
#i libri con "pochi prestiti". Come definisco "pochi prestiti"? Pur non avendo un'idea precisa, considerando che
#è richiesto di filtrare i libri "molto" poco letti e considerando che i libri nella top 10 si aggirano
#più o meno sui 1000 prestiti, filtrerei i libri con meno di 50 prestiti (?).

#Cerchiamo i libri con meno di 50 prestiti.

DFlessthan50 = DFfilterednomagazines.groupby("manifestation_id", "title").count()
DFlessthan50 = DFlessthan50.filter(DFlessthan50['count'] < 50)
DFlessthan50.sort("count", ascending=False).show()

#Ora usiamo un antijoin?

DFfilterednofewbooks = DFfilterednomagazines.join(DFlessthan50.select("manifestation_id"), DFlessthan50.manifestation_id == DFfilterednomagazines.manifestation_id, "leftanti")
DFfilterednofewbooks.show()

#Testiamo se l'antijoin ha funzionato con un libro a caso fra quelli con count = 49

DFfilterednofewbooks.filter(DFfilterednofewbooks.manifestation_id == "327518").show()
DFfilterednomagazines.filter(DFfilterednomagazines.manifestation_id == "327518").show()



+----------------+--------------------+-----+
|manifestation_id|               title|count|
+----------------+--------------------+-----+
|          336875|Tre volte invano ...|   49|
|          136727|La passione secon...|   49|
|          582300|Palloncino blu / ...|   49|
|           32334|L'uomo che sussur...|   49|
|           36131|Terrore dagli abi...|   49|
|          158328|Quanti impegni, S...|   49|
|          438378|Sun Tzu e l'arte ...|   49|
|          234699|La socializzazion...|   49|
|           34266|Il grande libro d...|   49|
|          116908|Ore giapponesi / ...|   49|
|           23908|Anna è furiosa / ...|   49|
|           83265|Maschere nude / L...|   49|
|           53149|Come si fa una te...|   49|
|          166989|Storia della filo...|   49|
|          904579|La mennulara : gr...|   49|
|          251122|Vietnam / Piero T...|   49|
|          919794|April è scomparsa...|   49|
|          101055|Tornano le rondin...|   49|
|          651062|Tranquillo prof,

In [27]:
#Sembra andare. Adesso controlliamo quanti prestiti sono rimasti in DFfilterednofewbooks (spero almeno più di 1000).
#Sembrano essere (se non ho fatto errori) 18647 contro i 257054 del set iniziale.

DFfilterednofewbooks.select("manifestation_id").distinct().groupby("manifestation_id").count().agg({"count": "sum"}).show()
DFfilteredloans.select("manifestation_id").distinct().groupby("manifestation_id").count().agg({"count": "sum"}).show()

+----------+
|sum(count)|
+----------+
|     18647|
+----------+

+----------+
|sum(count)|
+----------+
|    257054|
+----------+

